In [ ]:
import numpy as np
import pandas as pd
import pathlib
import os
from tqdm.auto import tqdm


from omegaconf import DictConfig, OmegaConf

import torch
#from torch_geometric import seed_everything

import ray

In [ ]:
base_path = "/home/jakobs"

project_path = f"{base_path}/data"

experiment = '231117'
experiment_path = f"{project_path}/{experiment}"
pathlib.Path(experiment_path).mkdir(parents=True, exist_ok=True)

partitions = [i for i in range(22)]

## Process Predictions

In [ ]:
endpoints_md = pd.read_feather(f"{base_path}/data/endpoints_md.feather").set_index("endpoint")
endpoints= endpoints_md.index.to_list()
endpoints_md

In [ ]:
id_vars = ["eid", "model", "partition", "split"]

In [ ]:
out_path = f"{experiment_path}/loghs"
pathlib.Path(out_path).mkdir(parents=True, exist_ok=True)

In [ ]:
data_baseline = pd.read_feather(f"{base_path}/data/data_baseline_231117.feather")

In [ ]:
data_outcomes_long = pd.read_feather(f"{base_path}/data/data_outcomes_long_230320.feather")

In [ ]:
data_outcomes_long.query("event==1").sample(10)

In [ ]:
endpoint_selection = [
    # generally very important
    "phecode_202", # Diabetes mellitus
    "phecode_401",	#Hypertension"  
    "phecode_404", # Ischemic heart disease   
    "phecode_404-1", # Myocardial infarction [Heart attack]
    "phecode_431-11", # Cerebral infarction [Ischemic stroke]
    "phecode_424", # Heart failure

    
    "phecode_059-1", # COVID 19
    "phecode_468", # Pneumonia
    "phecode_474", # Chronic obstructive pulmonary disease [COPD]
      
    "phecode_286-2", #	Major depressive disorder
    "phecode_324-11", #Parkinson's Disease
    "phecode_328", # Dementias and cerebral degeneration

    
    "phecode_164", # Anemia
    "phecode_726-1", # Osteoporosis
    "phecode_371", # Cataract
    "phecode_374-42", # Diabetic retinopathy
    "phecode_374-5", # Macular degeneration
    "phecode_375-1", # Glaucoma
    
    
    "phecode_103", # Malignant neoplasm of the skin
    "phecode_101", # Malignant neoplasm of the digestive organs
    "phecode_102", # LUNG CANCER
    
    "phecode_583", # Chronic kidney disease    
    "phecode_542", # Chronic liver disease and sequelae    
    "OMOP_4306655"] # All-Cause Death

In [ ]:
endpoint_frequencies = data_outcomes_long.query("event==1").value_counts("endpoint").to_frame("n_epic").merge(endpoints_md, how="left", left_index=True, right_index=True)

In [ ]:
endpoint_frequencies.query("endpoint==@endpoint_selection")

In [ ]:
endpoint_frequencies.query("n_epic>20&phecode_string==phecode_string").sort_index().reset_index().to_feather(f"{base_path}/data/endpoints_epic_md.feather")

In [ ]:
endpoints_md = pd.read_feather(f"{base_path}/data/endpoints_epic_md.feather").set_index("endpoint")
endpoints_md

In [ ]:
data_images = pd.read_feather(f"{base_path}/data/data_images_231117.feather").rename(columns={"distfilename":"img_name"})
data_images

In [ ]:
predictions = pd.read_feather(f"{base_path}/data/predictionstta_wide_231117.feather").sort_values("img_name")
predictions

In [ ]:
na_count = np.isnan(predictions[endpoints].values).sum()
all_count = predictions[endpoints].values.shape[0]*predictions[endpoints].values.shape[1]
na_count/all_count

In [ ]:
predictions_agg = predictions.groupby("img_name")[endpoints_md.index].mean().sort_index().reset_index()
predictions_agg

In [ ]:
data_predictions_all = data_baseline[["eid"]].merge(data_images, how="left", on="eid").merge(predictions_agg, how="left", on="img_name").reset_index(drop=True)

In [ ]:
data_predictions_all

In [ ]:
data_predictions_qc = data_predictions_all.query("quality!='Poor'").groupby("eid")[endpoints_md.index].mean()

In [ ]:
splits = pd.read_feather(f"{base_path}/data/splits_230321.feather")

In [ ]:
predictions_clean = splits.merge(data_predictions_qc, how="left", left_on="eid", right_index=True)

In [ ]:
predictions_clean

In [ ]:
predictions_clean.query("split=='test'").query("OMOP_4306655!=OMOP_4306655")#.isna().sum()# > 0).sum()

In [ ]:
model = "RetinaUKB"

for partition in range(10):
    temp_partition = predictions_clean.query("partition==@partition")
    for split in ["train", "test"]: 
        fp_out = f"{out_path}/{model}/{partition}"
        pathlib.Path(fp_out).mkdir(parents=True, exist_ok=True)
        t = temp_partition.query("split==@split").reset_index(drop=True)
        t.to_feather(f"{fp_out}/{split}.feather")
        print(f"{fp_out}/{split}.feather")

In [ ]:
out_path

In [ ]:
1+1